In [18]:
import pandas as pd
import datetime
import time

import sys
sys.path.append('..')
from utils.utils_iFEMG_feature import read_label

# muscle name VS ch nums
- ch1: RF, Rectus femoris, 股直肌
- ch2: LF, Lateral femoris, 股外侧肌
- ch3: SarT, sartorius, 缝匠肌
- ch4: BF, Biceps femoris, 股二头肌
- ch5: SemiT, semitendinosus, 半腱肌
- ch6: VM, Vastus medialis, 股内侧肌

In [ ]:
file_name_dict = {'aqian0':['affected', 'qian', '0'],
                  'aqian1_5':['affected', 'qian', '1.5'],
                  'aqian3':['affected', 'qian', '3'],
                  'ahou0':['affected', 'hou', '0'],
                  'ahou1_5':['affected', 'hou', '1.5'],
                  'ahou3':['affected', 'hou', '3'],
                  'hqian0':['healthy', 'qian', '0'],
                  'hqian1_5':['healthy', 'qian', '1.5'],
                  'hqian3':['healthy', 'qian', '3'],
                  'hhou0':['healthy', 'hou', '0'],
                  'hhou1_5':['healthy', 'hou', '1.5'],
                  'hhou3':['healthy', 'hou', '3']
                  }

In [23]:
folder_path = r"E:\fuckinggggg\paper_iFEMG_4th\data"
label_data = read_label(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b1" + '\\' + 'aqian0.txt')
b1_data = pd.read_table(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b1" + '\\' + 'aqian0.db', header=None, sep=';')
b2_data = pd.read_table(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b2" + '\\' + 'aqian0.db', header=None, sep=';')

In [ ]:
def alienMerge(b1_file: str, b2_file: str, label_file: str)->pd.DataFrame:
    '''
    将单次测试的数据通过时间戳对应，生成dataframe
    '''
    # 读取文件
    try:
        label_data = read_label(label_file)
        b1_data = pd.read_table(b1_file, header=None, sep=';')
        b2_data = pd.read_table(b2_file, header=None, sep=';')
    except FileNotFoundError:
        print(f"file {b1_file} not found")
        return 0
    #转换时间戳
    
    return

In [39]:
label_t_stamp = [] # 保存label文件中时间对应的时间戳，精度ms
for x in label_data:
    t = x[0] + " " + x[1]
    t_array = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S.%f").timestamp()
    #ret_stamp = int(time.mktime(t_array.timetuple()) * 1000 + t_array.microsecond/1000)
    label_t_stamp.append(t_array)

label_t_stamp

[1731569579.343986,
 1731569584.369697,
 1731569589.364151,
 1731569594.405721,
 1731569599.40898,
 1731569604.447724,
 1731569609.454855,
 1731569614.504423,
 1731569619.555754,
 1731569624.603994,
 1731569629.623484,
 1731569634.661786,
 1731569639.65352,
 1731569644.671694,
 1731569649.647895,
 1731569654.673263,
 1731569659.660728,
 1731569664.689167,
 1731569669.677253,
 1731569674.69825,
 1731569679.685412,
 1731569684.700191]

[1731569579343,
 1731569584369,
 1731569589364,
 1731569594405,
 1731569599408,
 1731569604447,
 1731569609454,
 1731569614504,
 1731569619555,
 1731569624603,
 1731569629623,
 1731569634661,
 1731569639653,
 1731569644671,
 1731569649647,
 1731569654673,
 1731569659660,
 1731569664689,
 1731569669677,
 1731569674698,
 1731569679685,
 1731569684700]

In [40]:
#将db文件中的时间转换为ms level时间戳
t_stamp = [] # 保存数据文件中时间转换的时间戳，精度ms
for t in b1_data[0].values:
    t_array = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S,%f").timestamp()

    t_stamp.append(t_array)
t_stamp

[1731569566.708,
 1731569566.708,
 1731569566.709,
 1731569566.709,
 1731569566.713,
 1731569566.713,
 1731569566.713,
 1731569566.713,
 1731569566.714,
 1731569566.714,
 1731569566.714,
 1731569566.714,
 1731569566.714,
 1731569566.715,
 1731569566.715,
 1731569566.715,
 1731569566.715,
 1731569566.716,
 1731569566.716,
 1731569566.716,
 1731569566.716,
 1731569566.716,
 1731569566.717,
 1731569566.717,
 1731569566.717,
 1731569566.718,
 1731569566.718,
 1731569566.718,
 1731569566.719,
 1731569566.719,
 1731569566.719,
 1731569566.72,
 1731569566.72,
 1731569566.72,
 1731569566.72,
 1731569566.72,
 1731569566.721,
 1731569566.721,
 1731569566.721,
 1731569566.721,
 1731569566.721,
 1731569566.721,
 1731569566.722,
 1731569566.722,
 1731569566.722,
 1731569566.722,
 1731569566.722,
 1731569566.722,
 1731569566.723,
 1731569566.723,
 1731569566.723,
 1731569566.723,
 1731569566.724,
 1731569566.724,
 1731569566.724,
 1731569566.724,
 1731569566.724,
 1731569566.725,
 1731569566.725,
 1

In [30]:
b1_data

,0,1,2,3,4,5,6,7,8,9
0,"2024-11-14 15:32:46,708",2991,0,88,0,1508,289,-891,721,NaN
1,"2024-11-14 15:32:46,708",2991,1469,88,3238,1237,283,-664,1091,NaN
2,"2024-11-14 15:32:46,709",2991,1469,88,3238,1398,284,-637,898,NaN
3,"2024-11-14 15:32:46,709",2991,1469,88,3230,957,285,-730,635,NaN
4,"2024-11-14 15:32:46,713",2991,1469,88,3230,945,284,-333,827,NaN
...,...,...,...,...,...,...,...,...,...,...
194292,"2024-11-14 15:34:47,808",3681,1486,424,3584,514,261,411,269,NaN
194293,"2024-11-14 15:34:47,808",3681,1486,424,3584,253,269,478,284,NaN
194294,"2024-11-14 15:34:47,809",3681,1486,424,3584,347,287,352,286,NaN
194295,"2024-11-14 15:34:47,809",3681,1486,424,3584,468,284,330,296,NaN


In [36]:
datetime.datetime.now().timestamp()

1733542594.398275